<figure> 
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Métodos ensamblados mixtos: Boosting</center></span>

<figure> 
<center>
<img src="../Imagenes/Mix_2_colors.jpg"  width="600" height="600" align="center"/>
<figcaption> </figcaption>
</center>
</figure>

Fuente <a href="https://commons.wikimedia.org/wiki/File:Mix_2_colors_(Unsplash).jpg">Pietro Jeng pietrozj</a>, CC0, via Wikimedia Commons

## <span style="color:blue">Referencias</span>

1. [Breiman, Friedman, Olsen, Stone, Classification and Regression Trees, 1984](http://library.lol/main/26908B6EDA02CA4FAF25ADBF57A12B26)
1. [Kumar, A. and Jain, M., Ensemble learning for AI developers](http://library.lol/main/AC20329F24A966566561C7BF2A2A8529)
1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2022](https://github.com/AprendizajeProfundo/Diplomado)


## <span style="color:blue">Autores</span>

1. Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
1. Daniel  Montenegro, Msc, dammontenegrore@unal.edu.co


## <span style="color:blue">Asesora de Medios y  Marketing</span>

1. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [](#)

## <span style="color:blue">Introducción</span>

En la primera parte de esta lección revisamos como entrenar diferentes máquinas de aprendizaje para el mismo conjunto de datos, para luego combinar los resultados usando diferentes técnicas en forma de votación y promediando.

En la segunda parte de la lección aprenderemos la técnica boosting, en la cual a partir de modelos en principio débiles es posibles obtener modelos más fuertes mejorando los previos mediante técnicas de mejora, impulso o boosting.

Esta lección esta basada en [scikit-learn Ensemble methods](https://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting) y [Kumar, A. and Jain, M., Ensemble learning for AI developers](http://library.lol/main/AC20329F24A966566561C7BF2A2A8529).

## <span style="color:blue">Algoritmo k-vecinos más cercanos para clasificación</span>

Adaptado de  de [IBM](https://www.ibm.com/topics/knn#:~:text=The%20k%2Dnearest%20neighbors%20algorithm%2C%20also%20known%20as%20KNN%20or,of%20an%20individual%20data%20point.).
El algoritmo de k-vecinos más cercanos, también conocido como KNN o k-NN, es un clasificador de aprendizaje supervisado no paramétrico, que utiliza la proximidad para hacer clasificaciones o predicciones sobre la agrupación de un punto de datos individual. Si bien se puede usar para problemas de regresión o clasificación, generalmente se usa como un algoritmo de clasificación, partiendo de la hipótesis de que se pueden encontrar puntos similares cerca uno del otro.


Para los problemas de clasificación, se asigna una etiqueta de clase sobre la base de un voto mayoritario, es decir, se utiliza la etiqueta que se representa con mayor frecuencia alrededor de un punto de datos determinado. Si bien esto se considera técnicamente "voto de la mayoría", el término "voto de la mayoría" se usa más comúnmente en la literatura. La distinción entre estas terminologías es que el "voto mayoritario" requiere técnicamente una mayoría superior al 50 %, lo que funciona principalmente cuando solo hay dos categorías. Cuando tiene varias clases, por ejemplo, cuatro categorías, no necesita necesariamente el 50% de los votos para llegar a una conclusión sobre una clase; puede asignar una etiqueta de clase con un voto superior al 25%. 

## <span style="color:blue">Métodos ensamblados por votación</span>

### <span style="color:#4CC9F0">Votación dura </span></span>

El conjunto de datos en el siguiente ejemplo se entrena usando tres modelos de aprendizaje automático: 

* k-vecinos más cercanos (KNN), 
* bosque aleatorio y 
* regresión logística

utilizando la biblioteca de Python   `scikit-learn`. Las salidas se combinan luego usando un clasificador de votación implementado en la biblioteca scikit-learn.

Si se mide la precisión resultante de cada uno de los modelos individuales, así como la
modelo ensamblado en el conjunto de datos de prueba, se obtiene un muy buena mejora en la precisión.

Ejemplo adaptado de [scikit-learn Ensemble methods](https://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting) y [Kumar, A. and Jain, M., Ensemble learning for AI developers](http://library.lol/main/AC20329F24A966566561C7BF2A2A8529), para los datos de cáncer de seno.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_breast_cancer
import numpy as np

# carga los datos
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.3, stratify=y, random_state=123)

### k-vecinos más cercanos
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
params_knn = {'n_neighbors': np.arange(1, 25)}
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(X_train, y_train)
knn_best = knn_gs.best_estimator_

### Clasificador bosques aleatorios
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
params_rf = {'n_estimators': [50, 100, 200]}
rf_gs = GridSearchCV(rf, params_rf, cv=5)
rf_gs.fit(X_train, y_train)
rf_best = rf_gs.best_estimator_

### Regresión Logística
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state=123,
solver='liblinear', penalty='l2', max_iter=5000)
C = np.logspace(1, 4, 10)
params_lr = dict(C=C)

lr_gs = GridSearchCV(log_reg, params_lr, cv=5, verbose=0)
lr_gs.fit(X_train, y_train)
lr_best = lr_gs.best_estimator_


# Combina los tres Conjuntos de Votación
from sklearn.ensemble import VotingClassifier
estimators=[('knn', knn_best), ('rf', rf_best), ('log_reg',
lr_best)]
ensemble = VotingClassifier(estimators, voting='hard')
ensemble.fit(X_train, y_train)
print("knn_gs.score: ", knn_best.score(X_test, y_test))
# salida: knn_gs.score:  0.9239766081871345
print("rf_gs.score: ", rf_best.score(X_test, y_test))
# salida: rf_gs.score:  0.9766081871345029
print("log_reg.score: ", lr_best.score(X_test, y_test))
#salida: log_reg.score:  0.9590643274853801
print("ensemble.score: ", ensemble.score(X_test, y_test))

# salida
# knn_gs.score:  0.9239766081871345
# rf_gs.score:  0.9766081871345029
# log_reg.score:  0.9707602339181286
# ensemble.score:  0.9766081871345029

### <span style="color:#4CC9F0">Modelos ensamblados por promedio: soft voting</span>

Promediar es otra forma de combinar la salida de diferentes clasificadores. La principal diferencia entre votar y promediar es que al promediar, tomamos la probabilidad de predicción de cada clase por separado del modelo y luego combine las probabilidades resultantes tomando el promedio de estos predicciones Este método de combinación se llama votación suave. Promediar es otra forma de combinar la salida de diferentes clasificadores.

La principal diferencia entre votar y promediar es que al promediar, tomamos la probabilidad de predicción de cada clase por separado del modelo y luego combine las probabilidades resultantes tomando el promedio de estos predicciones Este método de combinación se llama votación suave.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_breast_cancer
import numpy as np

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.3, stratify=y, random_state=0)

### k-Nearest Neighbors (k-NN)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
params_knn = {'n_neighbors': np.arange(1, 25)}
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(X_train, y_train)
knn_best = knn_gs.best_estimator_
knn_gs_predictions = knn_gs.predict(X_test)

### Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
params_rf = {'n_estimators': [50, 100, 200]}
rf_gs = GridSearchCV(rf, params_rf, cv=5)
rf_gs.fit(X_train, y_train)
rf_best = rf_gs.best_estimator_
rf_gs_predictions = rf_gs.predict(X_test)

### Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state=123,
solver='liblinear', penalty='l2', max_iter=5000)
C = np.logspace(1, 4, 10)
params_lr = dict(C=C)
lr_gs = GridSearchCV(log_reg, params_lr, cv=5, verbose=0)
lr_gs.fit(X_train, y_train)
lr_best = lr_gs.best_estimator_
log_reg_predictions = lr_gs.predict(X_test)

# combine all three by averaging the Ensembles results
average_prediction = (log_reg_predictions + knn_gs_predictions
+ rf_gs_predictions)/3.0
# Alternatively combine all through using VotingClassifier with voting='soft' parameter

# combine all three Voting Ensembles
from sklearn.ensemble import VotingClassifier
estimators=[('knn', knn_best), ('rf', rf_best), ('log_reg', lr_best)]
ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(X_train, y_train)

# salidas
print("knn_gs.score: ", knn_gs.score(X_test, y_test))

print("rf_gs.score: ", rf_gs.score(X_test, y_test))

print("log_reg.score: ", lr_gs.score(X_test, y_test))

print("ensemble.score: ", ensemble.score(X_test, y_test))

In [ ]:
#knn_gs.score:  0.9239766081871345
#rf_gs.score:  0.9532163742690059
#log_reg.score:  0.9415204678362573
#ensemble.score:  0.935672514619883

En el resto de la lección revisamos métodos mixtos.

## <span style="color:blue">Boosting</span>

Empezamos con un colección de modelos (`learners`). Cada learner de ML es entrenado  en un subconjunto particular de objetos de entrenamiento. Si un  modelo tiene un desempeño bajo, podríamos proporcionar mayor énfasis a ese alumno en particular. Esto se conoce como **boosting** (impulso).

### <span style="color:#4CC9F0">AdaBoost</span>

Primero, analicemos una de las estrategias  más simple pero más importantes de boosting,  `AdaBoost`. Empezamos con un colección de learners. Cada alumno de ML se entrena con un subconjunto particular de objetos de entrenamiento. Si un  modelo tiene un desempeño bajo, podríamos proporcionar mayor énfasis a ese modelo en particular. Esto se conoce como impulsar Primero, analicemos uno de los más simples pero más importantes de impulsar técnicas, AdaBoost.



<figure> 
<center>
<img src="../Imagenes/adaboost.png"  width="600" height="600" align="center"/>
<figcaption> </figcaption>
</center>
    
Fuente. Tomada de [Kumar, A. and Jain, M., Ensemble learning for AI developers](http://library.lol/main/AC20329F24A966566561C7BF2A2A8529)

El procedimiento es como sigue.

1. Inicialmente un modelo es entrenado con los datos originales. En la imagen, los puntos versee son bien clasificados y los rojos mal clasificados. Incialmente todos los datos tiene peso 1.
1. En el siguiente paso, a los los datos mal clasificados  se les incrementa el peso y se corre el entrenamiento de nuevo. Esta estrategia hace que el modelo preste más atención a esos datos con mayores pesos. Teóricamente, algunos objetos adicionales  quedan bien clasificados, pero otros permanecen más clasificados.
1. El paso anterior se repite a lograr la exactitud deseada.  El siguiente ejemplo muestra cómo usar AdaBoost con la biblioteca scikit-learn.

Ilustramos el procedimiento usando *scikit-learn* para los datos Iris. De forma predeterminada, *scikit-learn* utiliza un árbol de decisión como modelo básico, con profundidad máxima = 1. Para hacer un clasificador *AdaBoost*, pasamos un parámetro adicional, *n_estimadores* (en este ejemplo, n_estimadores = 100). *AdaBoost* se ejecuta en cada copia de pesos potenciada hasta que haya un ajuste de datos perfecto o el límite de *n_estimators* se alcance.

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier

X, y = load_iris(return_X_y=True)
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, X, y, cv=5)
print(scores.mean())

0.9466666666666665


Es importante entender que en cada paso del proceso boosting, un modelo que inicialmente puede ser muy débil es impulsado (mejorado), boosting, hacia un mejor modelo con el incremento de pesos en las observaciones mal clasificadas originalmente. ES decir, en cada paso se obtiene un modelo que hace mejor el trabajo de clasificación.

### <span style="color:#4CC9F0">Refuerzo del gradiente: Gradient Boosting</span>

El refuerzo de gradiente es similar a los métodos generales de boosting.  A diferencia de *AdaBoost*, en donde agrega un nuevo learner (modelo) después de aumentar el peso de las observaciones  mal clasificadas, en *gradient boosting*,  se entrena un nuevo modelo sobre los errores  residuales
cometidos por el predictor anterior.

### <span style="color:#4CC9F0">Refuerzo del gradiente: Gradient Boosting</span>

Basado en [Wikipedia](https://en.wikipedia.org/wiki/Gradient_boosting).

Al igual que otros métodos boosting, el `gradient boosting` combina "modelos" débiles en un solo modelo fuerte de manera iterativa. 

Es más fácil de explicar en la configuración de regresión de mínimos cuadrados , donde el objetivo es "entrenar" un modelo $\mathbf{F}$ para predecir valores de la forma $\hat{y} = \mathbf{F}(x)$ minimizando el error cuadrático medio $\tfrac{1}{n}\sum_{i}(\hat{y}_{i}-y_{i})^{2}$, para $n$ datos de entrenamiento $y_i$.

Ahora, consideremos un algoritmo gradient boosting con $M$ etapas $1 \leq m \leq M$. Supongamos algún modelo imperfecto $\mathbf{F}_m$. Para $m$ muy bajo el modelo regresa simplemente $\bar{y}_i = \bar{y}$.

 Para mejorar $\mathbf{F}_m$, nuestro algoritmo debería agregar algún nuevo estimador, $h_{m}(x)$, de tal modlo que  

$$
F_{m+1}(x_{i})=F_{m}(x_{i}) + h_{m}(x_{i}) = y_{i}
$$

o equivalente,

$$
 h_{m}(x_{i}) =  y_{i} - F_{m}(x_{i})
$$


Por lo tanto, el gradient boosting  $h_{m}$ al residual $y_{i}-F_{m}(x_{i})$. Como en otras variantes de boosting, cada$F_{m+1}$ intenta corregir los errores de su antecesor $F_m$. Se puede observar que los residuoles $h_{m}(x_{i})$  para un modelo dado son proporcionales a los gradientes negativos de la función de pérdida del error cuadrático medio (MSE) con respecto a $F(x_i)$:

$$
\begin{align}
L_{MSE} &= \frac{1}{n}\sum_{i=1}^{n}\left(y_{i}-F(x_{i})\right)^{2}\\
-\frac{\partial L_{MSE}}{\partial F(x_{i})} &= \frac{2}{n}(y_{i}-F( x_{i}))=\frac{2}{n}h_{m}(x_{i})
\end{align}
$$


Por lo tanto, el aumento de gradiente podría especializarse en un algoritmo de descenso de gradiente , y generalizarlo implica "conectar" a una pérdida diferente en su gradiente.

A continuación ilustramos gradient boost con ejemplo con datos sintéticos para clasificación generados con la función *make_hastie_10_2*.


In [22]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

X, y = make_hastie_10_2(random_state=0)

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X, y)
scores = cross_val_score(clf, X, y, cv=5)
print(scores.mean())

0.9225


### <span style="color:#4CC9F0">XGBoost</span>

XGBoost es un algoritmo y un sistema de software de última generación que se especializa en técnicas de aumento de gradiente residual. Mejora la técnica gradient boosting vainilla agregando los siguientes parámetros:


• Determina dinámicamente la profundidad de los árboles de decisión utilizados como aprendices débiles, añadiendo  parámetros de penalización  para la prevención de árboles con gran profundidad. Este evita el sobreajuste y mejora el rendimiento.
• Utiliza la reducción proporcional de los nodos de las hojas de los árboles.
• Utiliza el Newton's tree boosting  para optimizar el aprendizaje de estructuras de árboles.
• Agrega parámetros de aleatorización para un aprendizaje óptimo.

Puede consultar la documentación completa de XGBost [aquí](https://xgboost.readthedocs.io/en/latest/).

### <span style="color:#4CC9F0">Ejemplo de uso de XGBoost

 Para los detalles de uso de XGBoost lo remitimos al sitio oficial en Github [xgboost](https://github.com/dmlc/xgboost/tree/master/demo/CLI/regression). Es necesario instalar el producto con

In [ ]:
#!conda install -c conda-forge xgboost

En este ejemplo vamos a utilizar el conjunto de datos de `Pima Indians onset`  (inicio de diabetes de los indios Pima) .

Este conjunto de datos se compone de 8 variables de entrada que describen los detalles médicos de los pacientes y una variable de salida para indicar si el paciente tendrá un inicio de diabetes dentro de los 5 años.

Puede obtener más información sobre este conjunto de datos en el sitio web del repositorio de aprendizaje automático de UCI.

Este es un buen conjunto de datos para un primer modelo XGBoost porque todas las variables de entrada son numéricas y el problema es un problema de clasificación binaria simple. No es necesariamente un buen problema para el algoritmo XGBoost porque es un conjunto de datos relativamente pequeño y un problema fácil de modelar.

Datos descargados de [Kaggle](https://www.kaggle.com/datasets/kumargh/pimaindiansdiabetescsv?resource=download)

In [21]:
# Primer modelo  XGBoost para los datos Pima Indians dataset

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# datos
dataset = loadtxt('../Datos/pima-indians-diabetes.csv', delimiter=",")

# separa características (X)  y etiquetas (Y)
X = dataset[:,0:8]
Y = dataset[:,8]

# Divide los datos en entranamiento y prueba
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# ajusta el modelo sobre los datos de entrenamiento
model = XGBClassifier()
model.fit(X_train, y_train)

# hace predicciones para los datos de prueba
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evalúa las predicciones
accuracy = accuracy_score(y_test, predictions)

print(accuracy)

0.7401574803149606


In [ ]:
#